In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Conv2DTranspose, Concatenate, concatenate, AveragePooling2D, UpSampling2D
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

In [ ]:
# LOAD DATASETS

# dataset = np.load('dati/mist/datasets/dataset.npy')
# date = np.load('dati/mist/datasets/date.npy')
# baseline = np.load('dati/mist/datasets/baseline.npy')

date_d = np.load('dati/mist/datasets/date_d.npy')
date_n = np.load('dati/mist/datasets/date_n.npy')

dataset_d = np.load('dati/mist/datasets/dataset_d.npy')
dataset_n = np.load('dati/mist/datasets/dataset_n.npy')

italy_mask = np.load('dati/mist/datasets/italy_mask.npy')
data_min = np.load('dati/mist/datasets/data_min.npy')
data_max = np.load('dati/mist/datasets/data_max.npy')

abs_dataset_d = np.load('dati/mist/datasets/abs_dataset_d.npy')
abs_dataset_n = np.load('dati/mist/datasets/abs_dataset_n.npy')
# denorm_dataset_d = ((dataset_d + 1) / 2) * (data_max - data_min) + data_min
# denorm_dataset_n = ((dataset_n + 1) / 2) * (data_max - data_min) + data_min

In [ ]:
# ITERATION OVER DATASET - Check for problematic spots in real images

def calculate_spatial_variation(sample, treshold):
    """
    Calculate the spatial variation for each pixel by comparing it with its neighbors
    and return the average spatial variation for the sample.
    """
    height, width = sample.shape
    spatial_variations = np.zeros((height, width))
    max_variation_in_sample = 0
    high_variation_spots = []  # List to store coordinates of high variation spots
    
    for y in range(height):
        for x in range(width):
            pixel = sample[y, x]
            neighbors_values = sample[max(0, y-1):min(y+2, height), max(0, x-1):min(x+2, width)]
            max_diff = np.nanmax(np.abs(neighbors_values - pixel)) if np.count_nonzero(~np.isnan(neighbors_values)) > 0 else 0
            spatial_variations[y, x] = max_diff
            
            if max_diff > treshold:
                high_variation_spots.append((y, x))
                print(f"High spatial variation found at ({y}, {x}) with surrounding values:")
                print(sample[max(0, y-1):min(y+2, height), max(0, x-1):min(x+2, width)])
            
            if max_diff > max_variation_in_sample:
                max_variation_in_sample = max_diff

    # Calculate the average spatial variation for the sample
    average_spatial_variation = np.nanmean(spatial_variations)
    return average_spatial_variation, max_variation_in_sample, high_variation_spots

treshold = 7                        # Threshold considered high spatial variation
average_spatial_variations = []
samples_with_high_variation = 0
highest_spatial_variation = 0
dataset = abs_dataset_d             # Dataset to analyze

# Iterate over each sample in the dataset
for i in range(dataset.shape[0]):
    average_variation, max_variation_in_sample, high_variation_spots = calculate_spatial_variation(dataset[i, :, :], treshold)
    average_spatial_variations.append(average_variation)

    
    if max_variation_in_sample > treshold:
        samples_with_high_variation += 1
        # Plot the sample
        fig, ax = plt.subplots()
        ax.imshow(dataset[i, :, :], cmap='gray')
        # Mark all problematic spots
        for spot in high_variation_spots:
            rect = patches.Rectangle((spot[1]-0.5, spot[0]-0.5), 1, 1, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
        plt.show()

    if max_variation_in_sample > highest_spatial_variation:
        highest_spatial_variation = max_variation_in_sample

# Calculate the overall average spatial variation
overall_average_variation = np.mean(average_spatial_variations)
print(f"Overall average spatial variation: {overall_average_variation}")
print(f"Number of samples with spatial variation higher than {treshold}: {samples_with_high_variation} over {dataset.shape[0]}.")
print(f"Highest spatial variation in dataset: {highest_spatial_variation}")

In [ ]:
#Found using the difference between the average of the neighboring 8 pixels and the pixel value

i_d_3 = [50, 160, 184, 187, 205, 224, 227, 322, 364, 367, 406, 470, 488, 514, 526, 624, 686, 701, 725, 739, 850, 861, 865, 894, 903, 934, 936, 943, 952, 1026, 1029, 1051, 1053, 1074, 1078, 1083, 1085, 1094, 1142, 1193, 1278, 1287, 1293, 1299, 1357, 1361, 1375, 1471, 1479, 1614, 1641, 1661, 1777, 1793, 1797, 1799, 1825, 1851, 1867, 1869, 2208, 2339, 2351, 2357, 2366, 2503, 2510, 2526, 2548, 2549, 2560, 2563, 2613, 2730, 2743, 2872, 2885, 2895, 2915, 2916, 2930, 2933, 2938, 3063, 3066, 3072, 3104, 3112, 3224, 3261, 3273, 3291, 3407, 3446, 3448, 3473, 3474, 3502, 3503, 3505, 3506, 3547, 3609, 3620, 3626, 3631, 3654, 3716, 3801, 3905, 3997, 4014, 4028, 4032, 4034, 4036, 4047, 4151, 4162, 4166, 4171, 4172, 4178, 4180, 4281, 4283, 4284, 4316, 4329, 4345, 4402, 4411, 4434, 4553, 4655, 4708, 4709, 4740, 4753, 4756, 4764, 4771, 4916, 4944, 5065, 5087, 5136, 5196, 5254, 5268, 5270, 5280, 5282, 5295, 5309, 5310, 5436, 5449, 5458, 5471, 5477, 5622, 5637, 5642, 5746, 5802, 5808, 5809, 5820, 5822, 5831, 5848, 5854, 5856, 5968, 6011, 6014, 6015, 6025, 6041, 6171, 6191, 6218, 6221, 6234, 6239, 6240, 6270, 6351, 6352, 6357, 6358, 6384, 6443, 6498, 6562, 6569, 6578, 6589, 6590, 6700, 6704, 6726, 6754, 6755, 6783, 6882, 6904, 6931, 6961, 7068, 7093, 7105, 7118, 7135, 7280, 7286, 7313, 7318, 7320, 7322, 7338, 7373, 7394, 7427, 7437, 7440, 7462, 7470, 7485, 7595, 7601, 7623, 7628, 7642, 7644, 7647, 7653, 7659, 7674, 7678, 7696, 7778, 7782, 7783, 7786, 7799]  # Indexes with problematic spots above 3 in abs_dataset_d
dataset_d_3 = abs_dataset_d[i_d_3, :, :]
i_n_3 = [4, 9, 24, 40, 41, 44, 60, 63, 71, 95, 99, 101, 106, 109, 135, 151, 185, 187, 188, 189, 195, 202, 203, 205, 206, 211, 217, 218, 224, 227, 229, 230, 233, 236, 282, 311, 325, 336, 339, 343, 353, 355, 362, 366, 373, 382, 401, 406, 423, 431, 435, 437, 439, 440, 446, 456, 458, 462, 464, 471, 485, 495, 513, 523, 525, 527, 553, 562, 570, 574, 583, 617, 650, 666, 714, 721, 727, 730, 737, 745, 746, 748, 755, 771, 787, 796, 804, 821, 824, 830, 833, 839, 846, 849, 860, 865, 866, 888, 891, 893, 894, 895, 896, 897, 909, 910, 913, 914, 923, 926, 933, 936, 945, 973, 974, 1005, 1013, 1043, 1067, 1070, 1075, 1090, 1093, 1102, 1106, 1114, 1116, 1134, 1149, 1150, 1153, 1154, 1157, 1160, 1162, 1171, 1175, 1178, 1186, 1188, 1193, 1198, 1199, 1204, 1206, 1210, 1211, 1218, 1246, 1249, 1256, 1259, 1282, 1284, 1287, 1288, 1290, 1300, 1305, 1306, 1309, 1311, 1312, 1342, 1356, 1360, 1408, 1435, 1450, 1459, 1471, 1472, 1480, 1515, 1522, 1540, 1550, 1552, 1554, 1560, 1565, 1574, 1586, 1595, 1613, 1615, 1623, 1624, 1629, 1639, 1646, 1780, 1805, 1828, 1835, 1846, 1851, 1853, 1858, 1859, 1868, 1871, 1883, 1885, 1886, 1887, 1892, 1894, 1895, 1933, 1961, 1963, 1991, 1997, 2007, 2013, 2015, 2016, 2017, 2031, 2051, 2127, 2159, 2160, 2162, 2169, 2171, 2176, 2178, 2180, 2187, 2191, 2198, 2201, 2205, 2212, 2216, 2231, 2233, 2235, 2240, 2247, 2249, 2265, 2290, 2297, 2299, 2304, 2322, 2329, 2354, 2363, 2368, 2370, 2373, 2395, 2416, 2440, 2502, 2504, 2528, 2534, 2539, 2544, 2546, 2562, 2570, 2584, 2587, 2592, 2594, 2610, 2630, 2635, 2637, 2642, 2654, 2655, 2657, 2661, 2663, 2699, 2707, 2708, 2747, 2748, 2761, 2763, 2873, 2882, 2911, 2914, 2915, 2916, 2918, 2921, 2923, 2928, 2930, 2932, 2934, 2940, 2948, 2950, 2960, 2968, 2970, 2978, 2980, 2989, 2993, 3000, 3003, 3007, 3009, 3028, 3071, 3072, 3073, 3075, 3091, 3100, 3104, 3116, 3136, 3137, 3237, 3239, 3240, 3241, 3246, 3247, 3249, 3253, 3264, 3265, 3267, 3276, 3287, 3289, 3304, 3309, 3320, 3322, 3324, 3326, 3329, 3338, 3349, 3356, 3368, 3369, 3371, 3380, 3386, 3388, 3406, 3409, 3421, 3422, 3424, 3459, 3461, 3463, 3465, 3468, 3469, 3472, 3473, 3475, 3479, 3480, 3481, 3484, 3501, 3503, 3504, 3505, 3506, 3507, 3509, 3510, 3512, 3546, 3573, 3582, 3613, 3622, 3624, 3631, 3634, 3642, 3644, 3645, 3652, 3671, 3692, 3696, 3697, 3713, 3717, 3718, 3729, 3738, 3742, 3745, 3746, 3750, 3752, 3754, 3758, 3761, 3764, 3770, 3802, 3816, 3818, 3821, 3822, 3824, 3825, 3829, 3870, 3879, 3905, 3945, 3947, 3966, 3981, 3988, 3993, 3995, 3996, 4013, 4020, 4021, 4025, 4027, 4028, 4030, 4032, 4034, 4040, 4041, 4044, 4048, 4071, 4073, 4093, 4096, 4107, 4113, 4114, 4116, 4119, 4133, 4162, 4163, 4164, 4167, 4172, 4174, 4176, 4180, 4219, 4288, 4292, 4299, 4304, 4312, 4335, 4350, 4352, 4386, 4392, 4395, 4401, 4402, 4404, 4407, 4408, 4409, 4413, 4414, 4418, 4420, 4423, 4428, 4430, 4431, 4436, 4439, 4441, 4442, 4443, 4444, 4446, 4452, 4462, 4469, 4479, 4480, 4490, 4494, 4496, 4507, 4525, 4532, 4553, 4557, 4566, 4570, 4585, 4594, 4603, 4613, 4624, 4629, 4663, 4667, 4691, 4711, 4713, 4722, 4723, 4728, 4734, 4738, 4742, 4744, 4747, 4748, 4755, 4756, 4757, 4760, 4772, 4791, 4795, 4807, 4809, 4811, 4814, 4816, 4823, 4841, 4847, 4858, 4863, 4884, 4889, 4893, 4905, 4906, 4924, 4946, 4949, 4984, 5059, 5060, 5069, 5071, 5077, 5079, 5081, 5083, 5092, 5094, 5097, 5100, 5102, 5109, 5113, 5117, 5120, 5123, 5128, 5129, 5131, 5132, 5134, 5136, 5141, 5145, 5147, 5149, 5169, 5173, 5175, 5177, 5188, 5193, 5205, 5207, 5214, 5215, 5216, 5217, 5223, 5226, 5227, 5229, 5236, 5237, 5240, 5244, 5245, 5250, 5252, 5261, 5266, 5270, 5281, 5283, 5295, 5297, 5298, 5303, 5308, 5309, 5310, 5311, 5312, 5343, 5373, 5389, 5429, 5433, 5436, 5438, 5441, 5447, 5449, 5453, 5454, 5459, 5468, 5475, 5479, 5481, 5494, 5495, 5497, 5504, 5513, 5520, 5521, 5527, 5539, 5545, 5552, 5568, 5576, 5591, 5604, 5606, 5613, 5619, 5628, 5634, 5635, 5646, 5663, 5672, 5674, 5716, 5734, 5743, 5757, 5764, 5773, 5777, 5778, 5779, 5786, 5789, 5793, 5795, 5818, 5819, 5820, 5848, 5859, 5871, 5873, 5884, 5886, 5888, 5890, 5899, 5905, 5907, 5914, 5915, 5929, 5962, 5969, 5993, 6010, 6015, 6019, 6020, 6028, 6038, 6059, 6100, 6122, 6158, 6167, 6169, 6176, 6183, 6192, 6204, 6211, 6212, 6217, 6219, 6224, 6226, 6232, 6242, 6246, 6247, 6272, 6281, 6283, 6284, 6295, 6296, 6299, 6302, 6303, 6320, 6321, 6343, 6348, 6350, 6353, 6381, 6386, 6389, 6391, 6402, 6409, 6413, 6417, 6465, 6491, 6494, 6567, 6569, 6587, 6591, 6595, 6597, 6604, 6623, 6624, 6631, 6639, 6646, 6653, 6681, 6683, 6684, 6688, 6690, 6706, 6710, 6720, 6726, 6727, 6756, 6757, 6779, 6784, 6786, 6788, 6800, 6838, 6869, 6875, 6888, 6891, 6903, 6914, 6919, 6930, 6939, 6942, 6944, 6953, 6967, 6973, 6980, 6998, 7000, 7003, 7028, 7032, 7033, 7036, 7060, 7061, 7094, 7095, 7108, 7109, 7119, 7120, 7121, 7122, 7124, 7133, 7136, 7143, 7157, 7159, 7218, 7222, 7236, 7253, 7269, 7280, 7282, 7283, 7285, 7289, 7294, 7300, 7314, 7315, 7321, 7335, 7363, 7380, 7384, 7386, 7396, 7397, 7401, 7407, 7409, 7445, 7457, 7482, 7483, 7484, 7485, 7493, 7501, 7530, 7548, 7560, 7573, 7576, 7601, 7605, 7607, 7613, 7615, 7616, 7618, 7620, 7632, 7635, 7639, 7646, 7647, 7649, 7660, 7663, 7664, 7668, 7670, 7675, 7688, 7690, 7700, 7711, 7724, 7726, 7729, 7730, 7736, 7743, 7754, 7762, 7764, 7773, 7807, 7809]
dataset_n_3 = abs_dataset_n[i_n_3, :, :]

i_d_5 = [4740, 5087, 5136, 5436, 7653, 7678, 7696]  # Indexes with problematic spots above 5 in abs_dataset_d
dataset_d_5 = abs_dataset_d[i_d_5, :, :]
i_n_5 = [4494]  # Indexes with problematic spots above 5 in abs_dataset_n
dataset_n_5 = abs_dataset_n[i_n_5, :, :]

# =============================================================================
# Found using the max difference between the pixel and any of its neighboring 8 pixels

i_d_7 = [3066, 3224, 3905, 4740, 4754, 5087, 5136]
dataset_d_7 = abs_dataset_d[i_d_7, :, :]
i_n_7 = [533, 888, 910, 3104, 3468, 3504, 3507, 4494, 6019, 6020, 6381, 6389, 6756, 7773]
dataset_n_7 = abs_dataset_n[i_n_7, :, :]

In [ ]:
def calculate_spatial_variation(sample, threshold):
    """
    Calculate the spatial variation for each pixel by comparing it with the weighted average
    of its neighbors and return the average spatial variation for the sample.
    Differentiate between spots near NaNs and those without surrounding NaNs.
    Additionally, returns the number of problematic spots near NaNs and those not near NaNs.
    """
    height, width = sample.shape
    spatial_variations = np.zeros((height, width))
    max_variation_in_sample = 0
    high_variation_spots = []  # List to store coordinates of high variation spots and their type (near NaNs or not)
    near_nans_count = 0
    not_near_nans_count = 0
    
    for y in range(height):
        for x in range(width):
            pixel = sample[y, x]
            neighbors_values = sample[max(0, y-1):min(y+2, height), max(0, x-1):min(x+2, width)]
            valid_neighbors = neighbors_values[~np.isnan(neighbors_values)]
            
            if len(valid_neighbors) > 0:
                max_diff = np.nanmax(np.abs(neighbors_values - pixel)) if np.count_nonzero(~np.isnan(neighbors_values)) > 0 else 0
                spatial_variations[y, x] = max_diff
                # weighted_average = np.mean(valid_neighbors)
                # max_diff = np.abs(pixel - weighted_average)
                # spatial_variations[y, x] = max_diff
                
                if max_diff > threshold:
                    # Fetch a larger 5x5 area for printing
                    extended_neighbors_values = sample[max(0, y-2):min(y+3, height), max(0, x-2):min(x+3, width)]
                    print(f"High spatial variation found at ({y}, {x}) with surrounding 5x5 values:")
                    print(extended_neighbors_values)
                    # Check if the spot is near NaNs
                    is_near_nans = np.isnan(neighbors_values).any()
                    high_variation_spots.append(((y, x), is_near_nans))
                    if is_near_nans:
                        near_nans_count += 1
                    else:
                        not_near_nans_count += 1
                    
                if max_diff > max_variation_in_sample:
                    max_variation_in_sample = max_diff

    average_spatial_variation = np.nanmean(spatial_variations)
    return average_spatial_variation, max_variation_in_sample, high_variation_spots, near_nans_count, not_near_nans_count
    

dataset = abs_dataset_d # Dataset to analyze
threshold = 1           # Threshold considered (degrees considered high spatial variation)
average_spatial_variations = []
samples_with_high_variation = 0
highest_spatial_variation = 0
total_near_nans_count = 0
total_not_near_nans_count = 0
indexes_with_problematic_spots = []

for i in range(dataset.shape[0]):
    average_variation, max_variation_in_sample, high_variation_spots, near_nans_count, not_near_nans_count = calculate_spatial_variation(dataset[i, :, :], threshold)
    average_spatial_variations.append(average_variation)
    total_near_nans_count += near_nans_count
    total_not_near_nans_count += not_near_nans_count
    
    if high_variation_spots:  # Check if there are any problematic spots in the current sample
        samples_with_high_variation += 1
        indexes_with_problematic_spots.append(i)  # Add the index of the sample to the list
        
        fig, ax = plt.subplots()
        ax.imshow(dataset[i, :, :], cmap='viridis')
        #plt.colorbar()
        #plt.title(f"Sample {i}")
        for spot, is_near_nans in high_variation_spots:
            color = 'red' if is_near_nans else 'magenta'
            rect = patches.Rectangle((spot[1]-0.5, spot[0]-0.5), 1, 1, linewidth=2, edgecolor=color, facecolor='none')
            ax.add_patch(rect)
        plt.show()

    if max_variation_in_sample > highest_spatial_variation:
        highest_spatial_variation = max_variation_in_sample

overall_average_variation = np.mean(average_spatial_variations)
print(f"Number of samples with spatial variation higher than {threshold}: {samples_with_high_variation} over {dataset.shape[0]}.")
print(f"Number of problematic points near NaNs: {total_near_nans_count}")
print(f"Number of problematic points NOT near NaNs: {total_not_near_nans_count}")

print(f"Highest spatial variation in dataset: {highest_spatial_variation}")
print(f"Overall average spatial variation: {overall_average_variation}")

# Print the list of indexes with problematic spots
print(f"Indexes from dataset above {threshold}: {indexes_with_problematic_spots}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# PROF'S REQUEST TO MODIFY GRADIENT_1, 19/9/2024

def calculate_spatial_variation(sample, threshold):
    """
    Calculate the spatial variation for each pixel by comparing it with the weighted average
    of its neighbors and return the average spatial variation for the sample.
    Differentiate between spots near NaNs and those without surrounding NaNs.
    Additionally, returns the number of problematic spots near NaNs and those not near NaNs.
    """
    height, width = sample.shape
    spatial_variations = np.zeros((height, width))
    max_variation_in_sample = 0
    high_variation_spots = []  # List to store coordinates of high variation spots and their type (near NaNs or not)
    near_nans_count = 0
    not_near_nans_count = 0
    
    for y in range(height):
        for x in range(width):
            pixel = sample[y, x]
            neighbors_values = sample[max(0, y-1):min(y+2, height), max(0, x-1):min(x+2, width)]
            valid_neighbors = neighbors_values[~np.isnan(neighbors_values)]
            
            if len(valid_neighbors) > 0:
                max_diff = np.nanmax(np.abs(neighbors_values - pixel)) if np.count_nonzero(~np.isnan(neighbors_values)) > 0 else 0
                spatial_variations[y, x] = max_diff
                
                if max_diff > threshold:
                    # Fetch a larger 5x5 area for printing
                    extended_neighbors_values = sample[max(0, y-2):min(y+3, height), max(0, x-2):min(x+3, width)]
                    if not np.isnan(extended_neighbors_values).any():
                        # Since it's free from NaNs, it's considered problematic
                        high_variation_spots.append(((y, x), max_diff))  # Store the max_diff value
                        not_near_nans_count += 1
                        
                        if max_diff > max_variation_in_sample:
                            max_variation_in_sample = max_diff
                    
                if max_diff > max_variation_in_sample:
                    max_variation_in_sample = max_diff

    average_spatial_variation = np.nanmean(spatial_variations)
    return average_spatial_variation, max_variation_in_sample, high_variation_spots, near_nans_count, not_near_nans_count
    

dataset = abs_dataset_n  # Dataset to analyze
threshold = 1            # Threshold considered

# Analyze only sample 33
sample_index = 32
average_variation, max_variation_in_sample, high_variation_spots, near_nans_count, not_near_nans_count = calculate_spatial_variation(dataset[sample_index, :, :], threshold)

print(f"Sample {sample_index}:")
print(f"Date of the sample: {pd.to_datetime(date_n[sample_index], origin='unix', unit='D').strftime('%Y-%m-%d')}")

# Initialize figure and axis for the current sample
fig, ax = plt.subplots()
ax.imshow(dataset[sample_index, :, :], cmap='viridis')

# Plot points with different gradients
for spot, max_diff in high_variation_spots:
    if 1 < max_diff <= 1.5:
        color = 'magenta'  # Color for gradient between 1 to 1.5
    elif max_diff > 1.5:
        color = 'red'  # Color for gradient above 1.5
    rect = patches.Rectangle((spot[1]-0.5, spot[0]-0.5), 1, 1, linewidth=2, edgecolor=color, facecolor='none')
    ax.add_patch(rect)

plt.show()  # Show the figure after plotting all problematic spots for the sample

# Plot points with gradient between 1 to 1.5
fig, ax = plt.subplots()
ax.imshow(dataset[sample_index, :, :], cmap='viridis')
for spot, max_diff in high_variation_spots:
    if 1 < max_diff <= 1.5:
        color = 'magenta'  # Color for gradient between 1 to 1.5
        rect = patches.Rectangle((spot[1]-0.5, spot[0]-0.5), 1, 1, linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)

plt.show()  # Show the figure after plotting all problematic spots for the sample

# Plot points with gradient above 1.5
fig, ax = plt.subplots()
ax.imshow(dataset[sample_index, :, :], cmap='viridis')
for spot, max_diff in high_variation_spots:
    if max_diff > 1.5:
        color = 'red'  # Color for gradient above 1.5
        rect = patches.Rectangle((spot[1]-0.5, spot[0]-0.5), 1, 1, linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)

plt.show()  # Show the figure after plotting all problematic spots for the sample

In [ ]:
# direct, sensitive

def calculate_spatial_variation(sample, threshold):
    """
    Calculate the spatial variation for each pixel by comparing it with the weighted average
    of its neighbors and return the average spatial variation for the sample.
    Differentiate between spots near NaNs and those without surrounding NaNs.
    Additionally, returns the number of problematic spots near NaNs and those not near NaNs.
    """
    height, width = sample.shape
    spatial_variations = np.zeros((height, width))
    max_variation_in_sample = 0
    high_variation_spots = []  # List to store coordinates of high variation spots and their type (near NaNs or not)
    near_nans_count = 0
    not_near_nans_count = 0
    
    for y in range(height):
        for x in range(width):
            pixel = sample[y, x]
            neighbors_values = sample[max(0, y-1):min(y+2, height), max(0, x-1):min(x+2, width)]
            valid_neighbors = neighbors_values[~np.isnan(neighbors_values)]
            
            if len(valid_neighbors) > 0:
                max_diff = np.nanmax(np.abs(neighbors_values - pixel)) if np.count_nonzero(~np.isnan(neighbors_values)) > 0 else 0
                spatial_variations[y, x] = max_diff
                # weighted_average = np.mean(valid_neighbors)
                # max_diff = np.abs(pixel - weighted_average)
                # spatial_variations[y, x] = max_diff
                
                if max_diff > threshold:
                    # Fetch a larger 5x5 area for printing
                    extended_neighbors_values = sample[max(0, y-2):min(y+3, height), max(0, x-2):min(x+3, width)]
                    if not np.isnan(extended_neighbors_values).any():
                        print(f"High spatial variation found at ({y}, {x}) with surrounding 5x5 values:")
                        print(extended_neighbors_values)
                        # Since it's free from NaNs, it's considered problematic
                        high_variation_spots.append(((y, x), False))  # False indicates not near NaNs in the immediate neighbors
                        not_near_nans_count += 1
                        
                        if max_diff > max_variation_in_sample:
                            max_variation_in_sample = max_diff
                    
                if max_diff > max_variation_in_sample:
                    max_variation_in_sample = max_diff

    average_spatial_variation = np.nanmean(spatial_variations)
    return average_spatial_variation, max_variation_in_sample, high_variation_spots, near_nans_count, not_near_nans_count
    

dataset = abs_dataset_n # Dataset to analyze
threshold = 1           # Threshold considered
average_spatial_variations = []
samples_with_high_variation = 0
highest_spatial_variation = 0
total_near_nans_count = 0
total_not_near_nans_count = 0
indexes_with_problematic_spots = []

for i in range(dataset.shape[0]):
    average_variation, max_variation_in_sample, high_variation_spots, near_nans_count, not_near_nans_count = calculate_spatial_variation(dataset[i, :, :], threshold)
    average_spatial_variations.append(average_variation)
    total_near_nans_count += near_nans_count
    total_not_near_nans_count += not_near_nans_count
    
    if high_variation_spots:  # Check if there are any problematic spots in the current sample
        samples_with_high_variation += 1
        indexes_with_problematic_spots.append(i)  # Add the index of the sample to the list
        
        print(f"Sample {i}:")
        print(f"Date of the sample: {date_n[i]}")
        # Initialize figure and axis for the current sample
        fig, ax = plt.subplots()
        ax.imshow(dataset[i, :, :], cmap='viridis')
        #plt.colorbar()
        for spot, is_near_nans in high_variation_spots:
            if not is_near_nans:  # Plot only if the spot is not near NaNs
                color = 'magenta'  # Color for spots not near NaNs
                rect = patches.Rectangle((spot[1]-0.5, spot[0]-0.5), 1, 1, linewidth=2, edgecolor=color, facecolor='none')
                ax.add_patch(rect)
        
        plt.show()  # Show the figure after plotting all problematic spots for the sample

    if max_variation_in_sample > highest_spatial_variation:
        highest_spatial_variation = max_variation_in_sample

overall_average_variation = np.mean(average_spatial_variations)
print(f"Number of samples with spatial variation higher than {threshold}: {samples_with_high_variation} over {dataset.shape[0]}.")
print(f"Number of problematic points near NaNs: {total_near_nans_count}")
print(f"Number of problematic points NOT near NaNs: {total_not_near_nans_count}")

print(f"Highest spatial variation in dataset: {highest_spatial_variation}")
print(f"Overall average spatial variation: {overall_average_variation}")

# Print the list of indexes with problematic spots
print(f"Indexes from dataset above {threshold}: {indexes_with_problematic_spots}")